# In this notebook am using machine learning to create a model that predicts which passengers survived the Titanic shipwreck. Here is the [link](https://www.kaggle.com/c/titanic)

In [3]:
# modules for reading the data

import pandas as pd # data processing
import numpy as np # linear algebra

In [4]:
#getting the data and loading to pandas DataFrame

train_df = pd.read_csv('F://bizz//DATA SETS/TITANIC DATA SET/train.csv', index_col = 'PassengerId' ) # data to train our model
test_df = pd.read_csv('F://bizz//DATA SETS/TITANIC DATA SET/test.csv', index_col = 'PassengerId') #data to make predictions


In [5]:
#to have a feel of training data, this data will be used to train our model
train_df.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [6]:
#to have a feel of test data
#this data will be used to evaluate our model
test_df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,
892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [7]:
# Survived will act as our label on the testing data.
labl = train_df[ 'Survived']
type(labl)

pandas.core.series.Series

# LABEL is the thing we're predicting

In [8]:
#converting series object to dataframe, o means dint not survive 1 survived
y = pd.DataFrame(labl, index = train_df.index)
y.head()

,Survived
PassengerId,
1,0
2,1
3,1
4,1
5,0


In [9]:
#available features we only chose numerical
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 1 to 891
Data columns (total 11 columns):
Survived    891 non-null int64
Pclass      891 non-null int64
Name        891 non-null object
Sex         891 non-null object
Age         714 non-null float64
SibSp       891 non-null int64
Parch       891 non-null int64
Ticket      891 non-null object
Fare        891 non-null float64
Cabin       204 non-null object
Embarked    889 non-null object
dtypes: float64(2), int64(4), object(5)
memory usage: 66.1+ KB


# Creating a pipeline for numerical features only

In [10]:
from sklearn.pipeline import Pipeline
#Pipeline is used for tying together these dynamic parts of the ML process.
from sklearn.compose import ColumnTransformer
#transformations on the column ie choosing only numerical features
from sklearn.tree import DecisionTreeClassifier
#algorithm for predicting the results

In [11]:
# selecting columns to use
columns = ['Pclass', 'Parch', 'SibSp']

ct = ColumnTransformer(remainder = 'drop',
                       transformers = [
                           ('select', 'passthrough', columns)])

#creating the model
model_1 = Pipeline([
    ('selector', ct),
    ('predictor', DecisionTreeClassifier()),

])

In [12]:
#fitting/training the model
model_1.fit(train_df, y);

In [13]:
#making sure the test data and the train data have same columns

test_correct_columns = pd.DataFrame(test_df, columns=train_df.columns)


In [14]:
# custom fuction to make submissions

def make_submission(model, test_correct_columns):
    y_test_pred = model.predict(test_correct_columns)
    
    #predictions to dataframe
    predictions = pd.Series(data = y_test_pred,
                           index = test_df.index,
                           name = 'Survived')
    date = pd.Timestamp.now().strftime(format='%Y-%m-%d_%H-%M_')
    predictions.to_csv(f'{date}submission.csv', 
                       index=True, header=True)
    

In [15]:
#submission to local machine
make_submission(model_1, test_correct_columns)

In [16]:
#Evaluating the model performance
model_1.score(train_df, y)

0.7216610549943884

# MODEL PERFORMANCE
- The model on only three features('Pclass', 'Parch', 'SibSp')does not perfom badly given accuracy score of 72%
- Accuracy is used to measure the model performance
- Accuracy is equal to the number of observations you correctly classified over all observations. For example, if your model properly identified 77 out of 100 images, you have an accuracy of 77%
- We will also use PRECISION, RECALL and F1
- Precision is the fraction of true positives over all positive predictions. It is a measure of how "precise" our model was with regards to labeling observations as positive. Recall, on the other hand, is equal to the fraction of true positives over all positive observations. It is a measure of our model's ability to "catch" and properly label observations that are positive.
-  In addition to precision and recall, there is that  𝐹1  score which is the harmonic mean of precision and recall. It is a nice metric to use when we don't have a preference over precision and recall.

# 0.68 on kaggle(68%)

In [84]:
# Using cross_validation to score as kaggle
from sklearn.model_selection import cross_validate
result = cross_validate(model_1, train_df, y, cv=5) 
result['test_score'].mean()

0.6981733726696377

# model 2

# In model 2 am adding 2 features Sex and Embarked as they seem to affect survival 

In [85]:
from sklearn.preprocessing import OneHotEncoder
#converts the features into numbers ie male 1 female 0 
from sklearn.impute import SimpleImputer
#it fills the missing values ie in Embarked, machine learning models does not do well with missing values

#categorical features 
categorical_features = [ 'Sex', 'Embarked']

categorical_transformer = Pipeline([
    ('impute', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

# selecting columns to use (numerical)
columns = ['Pclass', 'Parch', 'SibSp']

ct = ColumnTransformer(remainder = 'drop',
                       transformers = [
                           ('select', 'passthrough', columns),
                           ('onehot', categorical_transformer, categorical_features)])

#creating the model
model_2 = Pipeline([
    ('selector', ct),
    ('predictor', DecisionTreeClassifier()),

])

In [86]:
model_2.fit(train_df, y);

In [87]:
make_submission(model_2, test_correct_columns)

In [88]:
model_2.score(train_df, y)

0.8372615039281706

In [89]:
# Using cross_validation to score
from sklearn.model_selection import cross_validate
result = cross_validate(model_2, train_df, y, cv=5) 
result['test_score'].mean()

0.7946205511267339

-Our model is doing well we get an improved score of  79%.
-The model is trained on five features('Pclass', 'Parch', 'SibSp', Sex, Embarked)

In [83]:
# Using cross_validation for f1, precision, recall
score = ['f1', 'precision', 'recall', 'accuracy']
from sklearn.model_selection import cross_validate
result = cross_validate(model_2, train_df, y, cv=5, scoring=score) 
result

{'fit_time': array([0.02498651, 0.02198839, 0.01999092, 0.01998973, 0.02098703]),
 'score_time': array([0.01898837, 0.02898264, 0.01598978, 0.01898932, 0.01499248]),
 'test_f1': array([0.65486726, 0.72580645, 0.73846154, 0.65517241, 0.68965517]),
 'test_precision': array([0.84090909, 0.80357143, 0.77419355, 0.79166667, 0.85106383]),
 'test_recall': array([0.53623188, 0.66176471, 0.70588235, 0.55882353, 0.57971014]),
 'test_accuracy': array([0.78212291, 0.80898876, 0.80898876, 0.7752809 , 0.79775281])}